In [1]:
pip install transformers sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import tensorflow as tf
import os
from transformers import BertTokenizer, TFBertForSequenceClassification


# Note that the `tpu` argument is for Colab-only
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.102.238.114:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.102.238.114:8470


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU')]


In [ ]:
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
with strategy.scope():
    model = TFBertForSequenceClassification.from_pretrained("bert-base-cased")
  
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)
    loss_fun = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
    model.compile(optimizer=optimizer, loss=loss_fun)


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
    
inputs = tokenizer("Hello, my dog is cute", return_tensors="tf")
inputs["labels"] = tf.reshape(tf.constant(0), (-1, 1))  # Batch size 1

outputs = model(inputs)
loss = outputs.loss
logits = outputs.logits

In [ ]:
model.config

BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

In [ ]:
outputs

TFSequenceClassifierOutput([('loss',
                             <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.59809864], dtype=float32)>),
                            ('logits',
                             <tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ 0.05223056, -0.14785877]], dtype=float32)>)])

In [2]:
from sentence_transformers import SentenceTransformer, util
import torch
import requests
from IPython.display import display, HTML

embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Corpus with example sentences
corpus = {'Open weather App' : "https://www.google.com/search?q=weather",
          'Order Pizza' : "https://www.dominos.com/en/?utm_source=google&utm_medium=loclist&utm_campaign=localmaps",
          'Open google maps' : "https://www.google.com/maps",
          'Text mother' : "770-710-7745",
}

corpus_embeddings = embedder.encode(list(corpus.keys()), convert_to_tensor=True)

# Query sentences:
queries = ['Tell me the weather', 'Im hungry', 'I forgot something', "Where do I go?"]


# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    print(util.semantic_search(query_embedding, corpus_embeddings, top_k = 1 ))


Downloading: 100%|██████████| 1.18k/1.18k [00:00<00:00, 1.22MB/s]
Downloading: 100%|██████████| 10.2k/10.2k [00:00<00:00, 5.05MB/s]
Downloading: 100%|██████████| 116/116 [00:00<00:00, 116kB/s]
Downloading: 100%|██████████| 39.3k/39.3k [00:00<00:00, 19.4MB/s]
Downloading: 100%|██████████| 349/349 [00:00<00:00, 70.4kB/s]
Downloading: 100%|██████████| 90.9M/90.9M [00:12<00:00, 7.33MB/s]
Downloading: 100%|██████████| 53.0/53.0 [00:00<00:00, 17.9kB/s]
Downloading: 100%|██████████| 112/112 [00:00<00:00, 109kB/s]
Downloading: 100%|██████████| 466k/466k [00:00<00:00, 4.31MB/s]
Downloading: 100%|██████████| 350/350 [00:00<00:00, 348kB/s]
Downloading: 100%|██████████| 13.2k/13.2k [00:00<00:00, 12.7MB/s]
Downloading: 100%|██████████| 232k/232k [00:00<00:00, 3.43MB/s]
Downloading: 100%|██████████| 190/190 [00:00<00:00, 94.1kB/s]


[[{'corpus_id': 0, 'score': 0.4927324652671814}]]
[[{'corpus_id': 1, 'score': 0.31687501072883606}]]
[[{'corpus_id': 3, 'score': 0.19732409715652466}]]
[[{'corpus_id': 3, 'score': 0.1577710658311844}]]


In [ ]:
from IPython.core.display import display_html
r = requests.get("https://www.google.com/search?q=weather")
r.text
display(HTML(r.text))